# Australia Fires

Enlace a la documentación:
https://pypi.org/project/PyMySQL/

In [14]:
import pandas as pd 
import pymysql.cursors


In [20]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

In [2]:
connection = pymysql.connect(host="34.229.137.164", 
                            port=25001,
                             user='student',
                             password='lebowsky',
                             database= 'australia_fires_2',
                             cursorclass=pymysql.cursors.DictCursor)

In [71]:
cursor= connection.cursor()

In [4]:
connection.open

True

In [6]:
#opcion clara
query= """SELECT TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_TYPE = 'BASE TABLE' AND TABLE_CATALOG='dbName'"""
pd.read_sql_query(query, connection)

,TABLE_NAME


In [5]:
con.execute("SHOW TABLES")
con.fetchall()

[{'Tables_in_australia_fires_2': 'fire_archive_M6_96619'},
 {'Tables_in_australia_fires_2': 'fire_archive_V1_96617'},
 {'Tables_in_australia_fires_2': 'fire_nrt_M6_96619'},
 {'Tables_in_australia_fires_2': 'fire_nrt_V1_96617'},
 {'Tables_in_australia_fires_2': 'student_findings'}]

In [6]:
query1 = """

SELECT * 
FROM fire_archive_M6_96619"""

data1 = pd.read_sql_query(query1, connection)
data1

,index,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,fire_type
0,0,-11.807,142.0583,313.0,1.0,1.0,2019-08-01,56,Terra,MODIS,48,6.3,297.3,6.6,D,0
1,1,-11.7924,142.0850,319.3,1.0,1.0,2019-08-01,56,Terra,MODIS,71,6.3,297.3,11.3,D,0
2,2,-12.8398,132.8744,311.6,3.1,1.7,2019-08-01,57,Terra,MODIS,42,6.3,298.7,23.1,D,0
3,3,-14.4306,143.3035,310.1,1.1,1.1,2019-08-01,57,Terra,MODIS,33,6.3,296.1,6.5,D,0
4,4,-12.4953,131.4897,310.3,4.0,1.9,2019-08-01,57,Terra,MODIS,36,6.3,298.8,27.6,D,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36006,36006,-28.0656,131.6334,312.7,1.2,1.1,2019-09-30,1644,Aqua,MODIS,85,6.3,292.9,14.2,N,0
36007,36007,-28.2625,128.5494,304.9,1.0,1.0,2019-09-30,1644,Aqua,MODIS,27,6.3,290.9,6.4,N,0
36008,36008,-28.2941,128.5123,301.7,1.0,1.0,2019-09-30,1644,Aqua,MODIS,43,6.3,290.4,5.5,N,0
36009,36009,-28.3001,128.5530,303.6,1.0,1.0,2019-09-30,1644,Aqua,MODIS,41,6.3,290.3,5.7,N,0


In [34]:
data1.version.value_counts()

6.3    36011
Name: version, dtype: int64

In [26]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36011 entries, 0 to 36010
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   index       36011 non-null  int64  
 1   latitude    36011 non-null  object 
 2   longitude   36011 non-null  float64
 3   brightness  36011 non-null  float64
 4   scan        36011 non-null  float64
 5   track       36011 non-null  float64
 6   acq_date    36011 non-null  object 
 7   acq_time    36011 non-null  int64  
 8   satellite   36011 non-null  object 
 9   instrument  36011 non-null  object 
 10  confidence  36011 non-null  int64  
 11  version     36011 non-null  float64
 12  bright_t31  36011 non-null  float64
 13  frp         36011 non-null  float64
 14  daynight    36011 non-null  object 
 15  fire_type   36011 non-null  int64  
dtypes: float64(7), int64(4), object(5)
memory usage: 4.4+ MB


In [7]:
query2 = """
SELECT *
FROM fire_archive_V1_96617"""
data2 = pd.read_sql_query(query2, connection)
data2

,index,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,fire_type
0,0,-34.45902,150.88040,326.5,0.60,0.71,2019-08-01,246,N,VIIRS,n,1,290.7,5.8,3
1,1,-31.70724,151.07191,367.0,0.56,0.69,2019-08-01,247,N,VIIRS,h,1,291.9,10.4,0
2,2,-31.39626,149.65253,325.2,0.68,0.74,2019-08-01,247,N,VIIRS,n,1,292.7,5.6,0
3,3,-30.39774,152.06432,347.8,0.48,0.65,2019-08-01,247,N,VIIRS,n,1,287.6,9.1,0
4,4,-31.20248,151.89766,350.7,0.50,0.66,2019-08-01,247,N,VIIRS,n,1,292.3,9.4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184773,184773,-32.37209,116.10032,296.1,0.48,0.40,2019-09-30,1702,N,VIIRS,n,1,284.8,1.2,0
184774,184774,-32.38958,116.11151,296.1,0.48,0.40,2019-09-30,1702,N,VIIRS,n,1,284.7,1.1,0
184775,184775,-33.12481,116.03968,299.9,0.47,0.40,2019-09-30,1702,N,VIIRS,n,1,285.0,1.3,0
184776,184776,-33.6348,122.92641,296.2,0.43,0.38,2019-09-30,1702,N,VIIRS,n,1,285.9,0.5,0


In [8]:
query3 = """
SELECT *
FROM fire_nrt_M6_96619"""
data3 = pd.read_sql_query(query3, connection)
data3

,index,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight
0,0,-14.281,143.636,323.9,1.7,1.3,2019-10-01,25,Terra,MODIS,70,6.0NRT,302.3,26.8,D
1,1,-14.284,143.532,343.5,1.7,1.3,2019-10-01,25,Terra,MODIS,90,6.0NRT,306.3,84.3,D
2,2,-14.302,143.706,320.2,1.7,1.3,2019-10-01,25,Terra,MODIS,30,6.0NRT,305.0,14.1,D
3,3,-14.283,143.652,320.4,1.7,1.3,2019-10-01,25,Terra,MODIS,57,6.0NRT,303.3,18.4,D
4,4,-14.285,143.521,349.4,1.7,1.3,2019-10-01,25,Terra,MODIS,94,6.0NRT,304.7,110.7,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183588,183588,-32.675,122.113,300.5,2.4,1.5,2020-01-11,1515,Terra,MODIS,19,6.0NRT,289.3,15.3,N
183589,183589,-16.708,129.646,300.5,1.0,1.0,2020-01-11,1645,Aqua,MODIS,16,6.0NRT,280.8,2.9,N
183590,183590,-16.747,129.653,301.8,1.0,1.0,2020-01-11,1645,Aqua,MODIS,23,6.0NRT,277.9,3.7,N
183591,183591,-32.484,121.319,306.5,1.4,1.2,2020-01-11,1650,Aqua,MODIS,68,6.0NRT,291.9,11.6,N


In [9]:
query4 = """
SELECT *
FROM fire_nrt_V1_96617"""
data4 = pd.read_sql_query(query4, connection)
data4

,index,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight
0,0,-42.69706,147.70634,333.6,0.57,0.43,2019-10-01,336,N,VIIRS,n,1.0NRT,293.4,4.0,D
1,1,-42.26889,147.31104,336.2,0.39,0.44,2019-10-01,336,N,VIIRS,n,1.0NRT,296.9,3.9,D
2,2,-40.85202,145.38068,327.9,0.46,0.47,2019-10-01,336,N,VIIRS,n,1.0NRT,295.3,3.7,D
3,3,-42.39329,147.47144,346.9,0.38,0.43,2019-10-01,336,N,VIIRS,n,1.0NRT,294.2,6.2,D
4,4,-42.69701,147.70584,334.1,0.56,0.43,2019-10-01,336,N,VIIRS,n,1.0NRT,293.5,4.1,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
956252,956252,-32.66628,122.15253,306.9,0.39,0.44,2020-01-11,1630,N,VIIRS,n,1.0NRT,288.6,0.8,N
956253,956253,-32.58616,123.39582,299.0,0.51,0.41,2020-01-11,1630,N,VIIRS,n,1.0NRT,287.4,0.7,N
956254,956254,-33.37853,115.94735,309.7,0.40,0.60,2020-01-11,1630,N,VIIRS,n,1.0NRT,291.7,1.0,N
956255,956255,-33.21827,115.75078,300.2,0.49,0.65,2020-01-11,1806,N,VIIRS,n,1.0NRT,290.2,1.1,N


In [28]:
data4.satellite.value_counts()

N    956257
Name: satellite, dtype: int64

In [22]:
data4.acq_date.value_counts()

2020-01-04    35038
2019-12-30    34193
2019-11-08    27223
2019-12-31    26085
2020-01-02    25128
              ...  
2020-01-10     2925
2019-10-02     2842
2019-11-01     2787
2019-11-03     2674
2019-10-31     1524
Name: acq_date, Length: 103, dtype: int64

In [21]:
data4.confidence.value_counts()

n    771035
h    108407
l     76815
Name: confidence, dtype: int64

In [10]:
query5 = """
SELECT *
FROM student_findings"""
data5 = pd.read_sql_query(query5, connection)
data5


,index,chosen_model,acc_score,recall_score


In [20]:
dict_data= {"data1":data1.columns.value_counts(), "data2": data2.columns.value_counts(), "data3": data3.columns.value_counts(), "data4": data4.columns.value_counts()}
dt_columns= pd.DataFrame(dict_data)
dt_columns.T

,acq_date,acq_time,bright_t31,bright_ti4,bright_ti5,brightness,confidence,daynight,fire_type,frp,index,instrument,latitude,longitude,satellite,scan,track,version
data1,1.0,1.0,1.0,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
data2,1.0,1.0,NaN,1.0,1.0,NaN,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
data3,1.0,1.0,1.0,NaN,NaN,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
data4,1.0,1.0,NaN,1.0,1.0,NaN,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [33]:
data4.isna().sum().sort_values(ascending=True)

index         0
latitude      0
longitude     0
bright_ti4    0
scan          0
track         0
acq_date      0
acq_time      0
satellite     0
instrument    0
confidence    0
version       0
bright_ti5    0
frp           0
daynight      0
dtype: int64

In [74]:
data1.corr()

,index,longitude,brightness,scan,track,acq_time,confidence,version,bright_t31,frp,fire_type
index,1.000000,1.037563e-01,1.581038e-01,-4.500031e-03,-5.544107e-03,5.437250e-02,7.548934e-02,0.000000e+00,9.719405e-02,9.474268e-02,-4.516212e-03
longitude,0.103756,1.000000e+00,-1.040974e-01,-5.184018e-02,-5.476704e-02,1.163678e-01,-3.552895e-02,-4.866327e-14,-3.804062e-01,4.492965e-04,5.580376e-03
brightness,0.158104,-1.040974e-01,1.000000e+00,-1.059401e-01,-1.085723e-01,-3.232710e-01,6.146908e-01,5.808995e-14,6.068445e-01,6.611308e-01,-5.416031e-02
scan,-0.004500,-5.184018e-02,-1.059401e-01,1.000000e+00,9.824843e-01,-2.687651e-02,-1.038080e-01,2.102575e-13,-9.435634e-02,2.921166e-01,-2.412955e-02
track,-0.005544,-5.476704e-02,-1.085723e-01,9.824843e-01,1.000000e+00,-2.439943e-02,-1.059490e-01,5.918211e-13,-9.206383e-02,2.865629e-01,-2.446029e-02
acq_time,0.054372,1.163678e-01,-3.232710e-01,-2.687651e-02,-2.439943e-02,1.000000e+00,-7.001169e-02,-8.797976e-15,-5.524480e-01,-7.365825e-02,7.623754e-03
confidence,0.075489,-3.552895e-02,6.146908e-01,-1.038080e-01,-1.059490e-01,-7.001169e-02,1.000000e+00,-5.660263e-15,2.385060e-01,3.553878e-01,-4.163203e-02
version,0.000000,-4.866327e-14,5.808995e-14,2.102575e-13,5.918211e-13,-8.797976e-15,-5.660263e-15,1.000000e+00,2.596921e-13,-6.018599e-15,-1.616750e-15
bright_t31,0.097194,-3.804062e-01,6.068445e-01,-9.435634e-02,-9.206383e-02,-5.524480e-01,2.385060e-01,2.596921e-13,1.000000e+00,2.168240e-01,-5.439344e-02
frp,0.094743,4.492965e-04,6.611308e-01,2.921166e-01,2.865629e-01,-7.365825e-02,3.553878e-01,-6.018599e-15,2.168240e-01,1.000000e+00,-2.668781e-02


## Entrenamiento don data 1 para predecir el fire_type en data 3

In [11]:
x_data1= data1.drop(columns=["index","acq_date", "acq_time", "satellite", "instrument", "version", "fire_type"])
x_data1

,latitude,longitude,brightness,scan,track,confidence,bright_t31,frp,daynight
0,-11.807,142.0583,313.0,1.0,1.0,48,297.3,6.6,D
1,-11.7924,142.0850,319.3,1.0,1.0,71,297.3,11.3,D
2,-12.8398,132.8744,311.6,3.1,1.7,42,298.7,23.1,D
3,-14.4306,143.3035,310.1,1.1,1.1,33,296.1,6.5,D
4,-12.4953,131.4897,310.3,4.0,1.9,36,298.8,27.6,D
...,...,...,...,...,...,...,...,...,...
36006,-28.0656,131.6334,312.7,1.2,1.1,85,292.9,14.2,N
36007,-28.2625,128.5494,304.9,1.0,1.0,27,290.9,6.4,N
36008,-28.2941,128.5123,301.7,1.0,1.0,43,290.4,5.5,N
36009,-28.3001,128.5530,303.6,1.0,1.0,41,290.3,5.7,N


In [76]:
x_data1.corr()

,longitude,brightness,scan,track,confidence,bright_t31,frp
longitude,1.000000,-0.104097,-0.051840,-0.054767,-0.035529,-0.380406,0.000449
brightness,-0.104097,1.000000,-0.105940,-0.108572,0.614691,0.606844,0.661131
scan,-0.051840,-0.105940,1.000000,0.982484,-0.103808,-0.094356,0.292117
track,-0.054767,-0.108572,0.982484,1.000000,-0.105949,-0.092064,0.286563
confidence,-0.035529,0.614691,-0.103808,-0.105949,1.000000,0.238506,0.355388
bright_t31,-0.380406,0.606844,-0.094356,-0.092064,0.238506,1.000000,0.216824
frp,0.000449,0.661131,0.292117,0.286563,0.355388,0.216824,1.000000


In [12]:
y_data1= data1["fire_type"]
y_data1.shape

(36011,)

Normalización latitud y longitud

In [15]:
lat= np.array(x_data1["latitude"]).reshape(-1,1)
log= np.array(x_data1["longitude"]).reshape(-1,1)
lat.shape, log.shape

((36011, 1), (36011, 1))

In [16]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() #Normaliza los datos de 0 a 1
scaler.fit(lat)
lat_normalized = scaler.transform(lat)
lat_normalized

array([[0.94694434],
       [0.94739096],
       [0.91535078],
       ...,
       [0.44260053],
       [0.44241699],
       [0.3639837 ]])

In [17]:
scaler.fit(log)
log_normalized = scaler.transform(log)
log_normalized

array([[0.70974278],
       [0.71042068],
       [0.47656661],
       ...,
       [0.36581434],
       [0.36684769],
       [0.18776167]])

In [18]:
dn= np.array(x_data1["daynight"]).reshape(-1,1)
dn.shape

(36011, 1)

Transform con label encoder dynight

In [21]:
le = preprocessing.LabelEncoder()
le.fit(dn)
dn= le.transform(dn)
dn

array([0, 0, 0, ..., 1, 1, 1])

In [22]:
X_1= x_data1.copy()
X_1["latitude"]= lat_normalized
X_1["longitude"]= log_normalized
X_1["daynight"]= dn
X_1

,latitude,longitude,brightness,scan,track,confidence,bright_t31,frp,daynight
0,0.946944,0.709743,313.0,1.0,1.0,48,297.3,6.6,0
1,0.947391,0.710421,319.3,1.0,1.0,71,297.3,11.3,0
2,0.915351,0.476567,311.6,3.1,1.7,42,298.7,23.1,0
3,0.866688,0.741358,310.1,1.1,1.1,33,296.1,6.5,0
4,0.925889,0.441410,310.3,4.0,1.9,36,298.8,27.6,0
...,...,...,...,...,...,...,...,...,...
36006,0.449590,0.445058,312.7,1.2,1.1,85,292.9,14.2,1
36007,0.443567,0.366756,304.9,1.0,1.0,27,290.9,6.4,1
36008,0.442601,0.365814,301.7,1.0,1.0,43,290.4,5.5,1
36009,0.442417,0.366848,303.6,1.0,1.0,41,290.3,5.7,1


## Parte ML


In [83]:
X_train, X_test, y_train, y_test = train_test_split(X_1, y_data1, test_size=0.2, random_state=2)

In [104]:
pipe = Pipeline(steps=[('classifier', RandomForestClassifier())])


logistic_params = {
    'classifier': [LogisticRegression()]}    
    

random_forest_params = {
    'classifier': [RandomForestClassifier()]
    }

svm_params = {
    'classifier': [SVC()]
    }
knn_params = {
    "classifier": [KNeighborsClassifier()]
}

XG= {
    "classifier": [XGBClassifier()]
}


# Create space of candidate learning algorithms and their hyperparameters
search_space = [
    logistic_params,
    random_forest_params,
    svm_params,
    knn_params, 
    XG
    ]

In [105]:
#cv = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1)#cross-validation?
# Create grid search 
clf = GridSearchCV(estimator=pipe, param_grid=search_space, verbose=0, n_jobs=-1, scoring="recall")

# Fit grid search
best_model = clf.fit(X_train, y_train)
# View best model
separator = "\n############################\n"
print(separator)
print("best estimator:", best_model.best_estimator_.get_params()['classifier'])
print(separator)
print("clf.best_params_", clf.best_params_)
print(separator)
# Mean cross-validated score of the best_estimator
print("clf.best_score", clf.best_score_)
#SAVE MODEL
# save the model to disk
#filename = 'finished_model.sav'
#pickle.dump(best_model, open(filename, 'wb'))


############################

best estimator: LogisticRegression()

############################

clf.best_params_ {'classifier': LogisticRegression()}

############################

clf.best_score nan


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_1, y_data1, test_size=0.2, random_state=2)

In [84]:
model = XGBClassifier()
model.fit(X_train, y_train)

[15:53:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [85]:
model.score(X_train,y_train)

1.0

To predict

In [27]:
def choose_x_y(df, col_name, col_name2, col_name3, col_name4, col_name5, col_name6, target_col=None):
    x_data= df.drop(columns=[col_name,col_name2, col_name3, col_name4, col_name5, col_name6])
    if target_col:
        y_data = df[target_col]
        return x_data, y_data
    else:
        return x_data

In [28]:
x_test_data= choose_x_y(df= data3,col_name= "index",col_name2="acq_date", col_name3="acq_time",col_name4= "satellite", col_name5="instrument", col_name6="version")
x_test_data

,latitude,longitude,brightness,scan,track,confidence,bright_t31,frp,daynight
0,-14.281,143.636,323.9,1.7,1.3,70,302.3,26.8,D
1,-14.284,143.532,343.5,1.7,1.3,90,306.3,84.3,D
2,-14.302,143.706,320.2,1.7,1.3,30,305.0,14.1,D
3,-14.283,143.652,320.4,1.7,1.3,57,303.3,18.4,D
4,-14.285,143.521,349.4,1.7,1.3,94,304.7,110.7,D
...,...,...,...,...,...,...,...,...,...
183588,-32.675,122.113,300.5,2.4,1.5,19,289.3,15.3,N
183589,-16.708,129.646,300.5,1.0,1.0,16,280.8,2.9,N
183590,-16.747,129.653,301.8,1.0,1.0,23,277.9,3.7,N
183591,-32.484,121.319,306.5,1.4,1.2,68,291.9,11.6,N


In [29]:
def encoder(label):
    le = preprocessing.LabelEncoder()
    le.fit(label)
    lb= le.transform(label)
    return lb

In [30]:
dyng= encoder(label= np.array(x_test_data["daynight"]).reshape(-1,1))

In [31]:
def normalize(x):
    scaler = MinMaxScaler() 
    scaler.fit(x)
    x_normalized = scaler.transform(x)
    return x_normalized

In [32]:
x_lat_test= normalize(x=np.array(x_test_data["latitude"]).reshape(-1,1))
x_log_test= normalize(x= np.array(x_test_data["longitude"]).reshape(-1,1))

In [86]:
X_pred= x_test_data.copy()
X_pred["latitude"]= x_lat_test
X_pred["longitude"]= x_log_test
X_pred["daynight"]= dyng
X_pred

,latitude,longitude,brightness,scan,track,confidence,bright_t31,frp,daynight
0,0.854902,0.754092,323.9,1.7,1.3,70,302.3,26.8,0
1,0.854813,0.751493,343.5,1.7,1.3,90,306.3,84.3,0
2,0.854280,0.755841,320.2,1.7,1.3,30,305.0,14.1,0
3,0.854843,0.754492,320.4,1.7,1.3,57,303.3,18.4,0
4,0.854784,0.751218,349.4,1.7,1.3,94,304.7,110.7,0
...,...,...,...,...,...,...,...,...,...
183588,0.309556,0.216272,300.5,2.4,1.5,19,289.3,15.3,1
183589,0.782946,0.404508,300.5,1.0,1.0,16,280.8,2.9,1
183590,0.781790,0.404683,301.8,1.0,1.0,23,277.9,3.7,1
183591,0.315218,0.196432,306.5,1.4,1.2,68,291.9,11.6,1


In [89]:
y_pred_xb= model.predict(X_pred)
y_pred_xb.shape

(183593,)

In [44]:
prueba= pd.DataFrame(y_pred)
prueba

,0
0,0
1,0
2,0
3,0
4,0
...,...
183588,0
183589,0
183590,0
183591,0


In [47]:
prueba= prueba.rename(columns={0:"y_pred"})
prueba.y_pred.value_counts()

0    183266
2       324
3         3
Name: y_pred, dtype: int64

In [90]:
score_xgb= model.score(X_pred,y_pred_xb)
recall_score(y_test,y_pred_xb, average="micro")

ValueError: Found input variables with inconsistent numbers of samples: [7203, 183593]

Random Forest Classifier

In [91]:
model_rfc= RandomForestClassifier()
model_rfc.fit(X_train, y_train)

RandomForestClassifier()

In [92]:
model_rfc.score(X_train, y_train)

1.0

In [61]:
y_pred_rdc= model_rfc.predict(X_test)

In [63]:
model_rfc.score(X_test,y_pred_test)

1.0

In [93]:
to_pred_rfc= model_rfc.predict(X_pred)
score_rfc= model.score(X_pred,to_pred_rfc)

In [94]:
recall_score(y_test, to_pred_rfc, average="micro")

ValueError: Found input variables with inconsistent numbers of samples: [7203, 183593]

In [69]:
score_xgb

1.0

In [77]:
score_rfc

0.9975380324957923

In [68]:
data5

,index,chosen_model,acc_score,recall_score


In [72]:
query_students = """
SELECT * FROM student_findings
"""
pd.read_sql_query(query_students, connection)

,index,chosen_model,acc_score,recall_score


In [76]:
cursor.execute("""
                UPDATE student_findings
                SET index = Mar
                WHERE chosen_model = 'XGBoost' and acc_score = 1.0 and recall_score = Nan
                """)

pd.read_sql_query(query_students, connection)

ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'index = Mar' at line 2")

In [97]:
cursor.execute(""" SHOW CREATE TABLE student_findings
""")
pd.read_sql_query(query_students, connection)

,index,chosen_model,acc_score,recall_score


In [102]:
cursor.execute("""
                INSERT INTO student_findings
                VALUES
                ("Mar", "XGBoost", 1.0, 0.0),
                ("Mar", "Random Forest Classifier", 0.997, 0.0)
                """)

pd.read_sql_query(query_students, connection)

IntegrityError: (1062, "Duplicate entry 'Mar' for key 'PRIMARY'")

In [95]:
from sklearn.metrics import recall_score
recall_score(y_test,y_pred_test, average="micro")

0.9969457170623351

In [53]:
from sklearn.metrics import logloss
model.logloss(y_test, y_pred)

ImportError: cannot import name 'logloss' from 'sklearn.metrics' (C:\Users\Usuario\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\__init__.py)

In [ ]:
bar = "######################################"
scores = ['precision', 'recall']
for score in scores:
    print(bar + "\n########## SCORE " + score + " ########### \n" + bar)
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(estimator=pipe, param_grid=search_space, verbose=0, n_jobs=-1, scoring=scores)
    best_model = clf.fit(X_train, y_train)
    # View best model
    separator = "\n############################\n"
    print(separator)
    print("best estimator:", best_model.best_estimator_.get_params()['classifier'])
    print(separator)
    print("clf.best_params_", clf.best_params_)
    print(separator)
    # Mean cross-validated score of the best_estimator
    print("clf.best_score", clf.best_score_)
#SAVE MODEL
# save the model to disk
#filename = 'finished_model.sav'
#pickle.dump(best_model, open(filename, 'wb'))